# Part 1

In [33]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests



In [44]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')

postal_data = []
columns = []
postal_codes_table = soup.find(class_='wikitable')
for index, tr in enumerate(postal_codes_table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    if (index == 0):
        columns = section
    else:
        postal_data.append(section)
        
canada_postal_data_df = pd.DataFrame(data = postal_data,columns = columns)
canada_postal_data_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [45]:
canada_postal_data_df = canada_postal_data_df[canada_postal_data_df.Borough!="Not assigned"]
print("Shape of Dataframe is - ",canada_postal_data_df.shape)
canada_postal_data_df.head()

Shape of Dataframe is -  (103, 3)


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [46]:
canada_postal_data_df['Neighborhood'].replace("Not assigned", canada_postal_data_df["Borough"],inplace=True)
canada_postal_data_df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [48]:
canada_postal_data_df['new_neighbor'] = np.where(canada_postal_data_df['Neighborhood']=='Not assigned',canada_postal_data_df['Borough'],canada_postal_data_df['Neighborhood'])
canada_postal_codes_df = pd.DataFrame(canada_postal_data_df.groupby(['Postal Code','Borough'])['new_neighbor'].apply(','.join).reset_index())
canada_postal_codes_df = canada_postal_codes_df.rename(columns = {"Postal Code": "PostalCode","new_neighbor":"Neighborhood"})

In [49]:
print(canada_postal_codes_df.head())
print("\n The Shape of the dataframe is: ",canada_postal_codes_df.shape)

  PostalCode      Borough                            Neighborhood
0        M1B  Scarborough                          Malvern, Rouge
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2        M1E  Scarborough       Guildwood, Morningside, West Hill
3        M1G  Scarborough                                  Woburn
4        M1H  Scarborough                               Cedarbrae

 The Shape of the dataframe is:  (103, 3)


# Part 2

In [ ]:
!conda install -c conda-forge geocoder --yes
import geocoder

In [35]:
import io


In [53]:
url = "http://cocl.us/Geospatial_data"
lat_long = requests.get(url).text
lat_long_df = pd.read_csv(io.StringIO(lat_long))
lat_long_df = lat_long_df.rename(columns= {'Postal Code':'PostalCode'})

lat_long_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [55]:
Toronto_data = pd.merge(canada_postal_codes_df,lat_long_df, on = 'PostalCode')
Toronto_data = Toronto_data.rename(columns= {'Neighbourhood':'Neighborhood'})
Toronto_data.head(12)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Part 3

In [57]:
from geopy.geocoders import Nominatim

In [58]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude_loc = location.latitude
longitude_loc = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude_loc, longitude_loc))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [67]:
!pip install folium
import folium

     |████████████████████████████████| 102kB 6.4MB/s ta 0:00:011


In [71]:
Toronto_Map = folium.Map(location=[latitude_loc, longitude_loc], zoom_start=10)
Toronto_Map

In [ ]:
for lat, lng, borough, neighborhood in zip(
        Toronto_data['Latitude'], 
        Toronto_data['Longitude'], 
        Toronto_data['Borough'], 
        Toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  